In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
import time

In [72]:
df = pd.read_csv("Fife-data/flex_networks.csv")
df["Timestamp"] = pd.to_datetime(df["Timestamp"], infer_datetime_format=True)
loads = np.asarray(df["crawfordCrescent_F2"])

In [73]:
class Battery:
    def __init__(self, maxSOC, maxChargeRate, maxDischargeRate, chargeEfficiency, dischargeEfficiency):
        self.maxSOC = maxSOC
        self.maxChargeRate = maxChargeRate
        self.maxDischargeRate = maxDischargeRate
        self.chargeEfficiency = chargeEfficiency
        self.dischargeEfficiency = dischargeEfficiency


h = 48
d = 161
day = loads[d*h:d*h+h]
peakLoad = max(loads)

SOC = max(loads)*0.25
D = max(loads)*0.25
C = D/2
efficiency = 0.95


batt = Battery(SOC, C, D, efficiency, efficiency)

In [74]:
loadDict = dict(enumerate(day))

In [75]:
# Initialise model
m = ConcreteModel()

# Create time to be used as index
m.Time = RangeSet(0, h-1)

In [76]:
# Declare Decision variables
m.SOC = Var(m.Time, bounds=(0,batt.maxSOC), initialize=0) # State of Charge variable, cant be greater than max SOC
m.posDeltaSOC = Var(m.Time, initialize=0) # Change of State of Charge
m.negDeltaSOC = Var(m.Time, initialize=0)
m.posEtoBatt = Var(m.Time, bounds=(0,batt.maxChargeRate*(30/60)), initialize=0) # Energy in grid, converted to Watts
m.negEfromBatt = Var(m.Time, bounds=(0,batt.maxDischargeRate*(30/60)), initialize=0)
m.netLoad = Var(m.Time, initialize=loadDict) # Net load from grid




In [77]:
# Boolean variables, used to determine wether battery is charging or discharging
m.Bool_char = Var(m.Time, within=Boolean) # 1 if battery is charging
m.Bool_dis= Var(m.Time, within=Boolean, initialize=0) # 1 if battery is discharging

In [78]:
m.dayLoads = Param(m.Time, initialize=loadDict)

In [79]:
m.chargeEfficiency = Param(initialize=batt.chargeEfficiency)
m.dischargeEfficiency = Param(initialize=batt.dischargeEfficiency)
m.chargingLimit = Param(initialize=batt.maxChargeRate*(30/60))
m.dischargingLimit = Param(initialize=batt.maxDischargeRate*(30/60))

In [80]:
# Objective function is max of loads in the day
def Obj_func(m):
    return max(list(m.netLoad.values()))
m.max_load = Objective(rule=Obj_func, sense=minimize)

ERROR: Rule failed when generating expression for Objective max_load with
    index None: PyomoException: Cannot convert non-constant Pyomo expression
    (netLoad[0]  <  netLoad[1]) to bool. This error is usually caused by using
    a Var, unit, or mutable Param in a Boolean context such as an "if"
    statement, or when checking container membership or equality. For example,
        >>> m.x = Var() >>> if m.x >= 1: ...     pass
    and
        >>> m.y = Var() >>> if m.y in [m.x, m.y]: ...     pass
    would both cause this exception.
ERROR: Constructing component 'max_load' from data=None failed:
    PyomoException: Cannot convert non-constant Pyomo expression (netLoad[0]
    <  netLoad[1]) to bool. This error is usually caused by using a Var, unit,
    or mutable Param in a Boolean context such as an "if" statement, or when
    checking container membership or equality. For example,
        >>> m.x = Var() >>> if m.x >= 1: ...     pass
    and
        >>> m.y = Var() >>> if m.y in [

PyomoException: Cannot convert non-constant Pyomo expression (netLoad[0]  <  netLoad[1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

In [ ]:
# Constraints
# SOC is equal to SOC at previous time plus change in SOC
def SOC_rule(m,t):
    if t==0:
        return(m.SOC[t] == m.posDeltaSOC[t] + m.negDeltaSOC[t])
    else:
        return(m.SOC[t] == m.SOC[t-1] + m.posDeltaSOC[t] + m.negDeltaSOC[t])
m.Batt_SOC = Constraint(m.Time, rule=SOC_rule)

In [ ]:
def Bool_char_rule_1(m,t):
    bigM = 500000
    return(m.posDeltaSOC[t] >= -bigM*(m.Bool_char[t]))
m.Batt_char1 = Constraint(m.Time, rule=Bool_char_rule_1)

def Bool_char_rule_2(m,t):
    bigM = 500000
    return(m.posDeltaSOC[t] <= 0+bigM*(1 - m.Bool_dis[t]))
m.Batt_char2 = Constraint(m.Time, rule=Bool_char_rule_2)

def Bool_char_rule_3(m,i):
    bigM = 500000
    return((m.negDeltaSOC[i]) <= bigM*(m.Bool_dis[i]))
m.Batt_cd3 = Constraint(m.Time,rule=Bool_char_rule_3)

def Bool_char_rule_4(m,i):
    bigM=500000
    return((m.negDeltaSOC[i]) >= 0-bigM*(1-m.Bool_char[i]))
m.Batt_cd4 = Constraint(m.Time,rule=Bool_char_rule_4)

def Bool_char_dis(m,t):
    return(m.Bool_char[t] + m.Bool_dis[t], 1)
m.Batt_char_dis = Constraint(m.Time, rule=Bool_char_dis)

In [ ]:
# Account for charging efficiency
def pos_E_in_rule(m,t):
    return m.posEtoBatt[t] == m.posDeltaSOC[t]/m.chargeEfficiency
m.posEIn_cons = Constraint(m.Time, rule=pos_E_in_rule)

# Account for discharging efficiency
def neg_E_out_rule(m,t):
    return m.negEfromBatt[t] == m.negDeltaSOC[t]*m.dischargeEfficiency
m.negEOut_cons = Constraint(m.Time, rule=neg_E_out_rule)

    'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [ ]:
# ensure charging rate obeyed
def E_charging_rate_rule(m,t):
    return m.posEtoBatt[t] <= m.chargingLimit
m.chargingLimit_cons = Constraint(m.Time, rule=E_charging_rate_rule)

# ensure DIScharging rate obeyed
def E_discharging_rate_rule(m,t):
    return m.negEfromBatt[t] >= m.dischargingLimit
m.dischargingLimit_cons = Constraint(m.Time, rule=E_discharging_rate_rule)

In [ ]:
# calculate the net positive demand
def E_net_rule(m,t):
    return m.netLoad[t] == m.dayLoads[t] + m.posEtoBatt[t] + m.negEfromBatt[t]
m.E_posNet_cons = Constraint(m.Time, rule=E_net_rule)

    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [ ]:
opt = SolverFactory("glpk", executable="D:\\glpk-4.65\\w64\\glpsol")

In [ ]:
t = time.time()
results = opt.solve(m)
elapsed = time.time() - t
print ('Time elapsed:', elapsed)